In [1]:
import requests
from bs4 import BeautifulSoup
import psycopg2

In [2]:
DB_NAME = "kitap_db"
DB_USER = "postgres"
DB_PASSWORD = "kerimcan1234"
DB_HOST = "localhost"
DB_PORT = "5432"

In [3]:
con = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port = DB_PORT
)
con.autocommit = True
cursor = con.cursor()

In [4]:
query_create_table = """ CREATE TABLE IF NOT EXISTS dr_kitaplar(
    id Serial PRIMARY KEY,
    kitapAdi TEXT NOT NULL,
    yazar TEXT NOT NULL,
    yayinEvi TEXT NOT NULL,
    ucret TEXT NOT NULL,
    link TEXT NOT NULL
    );
    """
cursor.execute(query_create_table)

In [5]:
def kullanici_ekle(a, b, c, d, e):
    query_INSERT = f"INSERT INTO dr_kitaplar (kitapAdi, yazar, yayinEvi, ucret, link) SELECT '{a}', '{b}', '{c}', '{d}', '{e}' WHERE NOT EXISTS ( SELECT * FROM dr_kitaplar WHERE kitapAdi='{a}' and yazar='{b}' and yayinEvi='{c}' and ucret='{d}' and link='{e}');"
    cursor.execute(query_INSERT)

In [6]:
header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36"
}

urunSayisi = 0
for sayfa in range(1, 61):
    r = requests.get(f"https://www.dr.com.tr/kategori/Kitap/Bilim/grupno=00052?Page={sayfa}", headers=header)

    soup = BeautifulSoup(r.content, "lxml")

    drKitaplar = soup.find_all("div", attrs={"class":"prd-content"})


    for kitaplar in drKitaplar:
        urunSayisi += 1
        kitapAd = kitaplar.find("a", attrs={"class": "prd-name"}).text.replace("'"," ")
        try:
            kitapYazar = kitaplar.find("a", attrs={"class": "who text-overflow"}).text.strip().replace("'"," ")
        except Exception:
            kitapYazar = kitaplar.find("div", attrs={"class": "prd-row"}).text.replace("'"," ")

        kitapYayin = kitaplar.find("a", attrs = {"class":"prd-publisher"}).text.strip()
        kitapUcret = kitaplar.find("div", attrs = {"class": "prd-price"}).text.strip()
        kitapURL =  "https://www.dr.com.tr" + kitaplar.find("a").get("href")
            
        kitap = {"Kitap-Adi":kitapAd, "Kitap-Yazar": kitapYazar, "Kitap-Yayin": kitapYayin, "Kitap-Ucret": kitapUcret, "Kitap-URL": kitapURL}

        kullanici_ekle(kitap["Kitap-Adi"], kitap["Kitap-Yazar"], kitap["Kitap-Yayin"], kitap["Kitap-Ucret"], kitap["Kitap-URL"])
print(f"toplam {urunSayisi} ürün başarılı bir şekilde veri tabanına eklendi.")

toplam 2700 ürün başarılı bir şekilde veri tabanına eklendi.
